## Questions to Answer

1. Question 1 - Visual Type A
2. Question 2 - Visual Type A
3. Question 3 - Visual Type B

4. Question 4 - Visual Type A
5. Question 5 - Visual Type A
6. Question 6 - Visual Type B

7. Question 7 - Visual Type A
8. Question 8 - Visual Type A
9. Question 9 - Visual Type B


In [1]:
import pandas as pd

YearinBusinessDF = pd.read_csv('ABSCS2018.AB1800CSA02-2023-04-20T164944.csv')
ReceiptSizeOfFirmDF = pd.read_csv('ABSCS2018.AB1800CSA03-2023-04-20T164951.csv')
EmploymentSizeOfFirmDF = pd.read_csv('ABSCS2018.AB1800CSA04-2023-04-20T164954.csv')

In [14]:
YearinBusinessDF.head()

,Geographic Area Name (NAME),2017 NAICS code (NAICS2017),Meaning of NAICS code (NAICS2017_LABEL),Meaning of Sex code (SEX_LABEL),Meaning of Ethnicity code (ETH_GROUP_LABEL),Meaning of Race code (RACE_GROUP_LABEL),Meaning of Veteran code (VET_GROUP_LABEL),Meaning of Years in business code (YIBSZFI_LABEL),Year (YEAR),Number of employer firms (FIRMPDEMP),"Sales, value of shipments, or revenue of employer firms ($1,000) (RCPPDEMP)",Number of employees (EMP),"Annual payroll ($1,000) (PAYANN)",Relative standard error of employer firms (%) (FIRMPDEMP_S),"Relative standard error of sales, value of shipments, or revenue of employer firms (%) (RCPPDEMP_S)",Relative standard error of number of employees (%) (EMP_S),Relative standard error of annual payroll (%) (PAYANN_S)
0,United States,00,Total for all sectors,Total,Total,Total,Total,Firms with less than 2 years in business,2018,"826,575","680,812,490","3,554,392","142,887,616",0.7,2.1,2.4,1.9
1,United States,00,Total for all sectors,Total,Total,Total,Total,Firms with 2 to 3 years in business,2018,"718,376","769,496,008","5,116,799","173,356,299",1.0,4.0,3.6,2.4
2,United States,00,Total for all sectors,Total,Total,Total,Total,Firms with 4 to 5 years in business,2018,"542,963","724,662,955","4,602,804","176,253,624",0.8,5.4,4.3,5.8
3,United States,00,Total for all sectors,Total,Total,Total,Total,Firms with 6 to 10 years in business,2018,"948,138","1,735,022,260","9,572,368","416,994,321",0.6,5.5,0.9,5.2
4,United States,00,Total for all sectors,Total,Total,Total,Total,Firms with 11 to 15 years in business,2018,"812,155","1,968,981,291","9,098,324","445,296,811",0.8,6.1,2.1,4.2


In [13]:
list(YearinBusinessDF.columns)

['Geographic Area Name (NAME)',
 '2017 NAICS code (NAICS2017)',
 'Meaning of NAICS code (NAICS2017_LABEL)',
 'Meaning of Sex code (SEX_LABEL)',
 'Meaning of Ethnicity code (ETH_GROUP_LABEL)',
 'Meaning of Race code (RACE_GROUP_LABEL)',
 'Meaning of Veteran code (VET_GROUP_LABEL)',
 'Meaning of Years in business code (YIBSZFI_LABEL)',
 'Year (YEAR)',
 'Number of employer firms (FIRMPDEMP)',
 'Sales, value of shipments, or revenue of employer firms ($1,000) (RCPPDEMP)',
 'Number of employees (EMP)',
 'Annual payroll ($1,000) (PAYANN)',
 'Relative standard error of employer firms (%) (FIRMPDEMP_S)',
 'Relative standard error of sales, value of shipments, or revenue of employer firms (%) (RCPPDEMP_S)',
 'Relative standard error of number of employees (%) (EMP_S)',
 'Relative standard error of annual payroll (%) (PAYANN_S)']

Columns to keep from YearinBusinessDF:

 'Geographic Area Name (NAME)',
 '2017 NAICS code (NAICS2017)',
 'Meaning of NAICS code (NAICS2017_LABEL)',
 'Meaning of Sex code (SEX_LABEL)',
 'Meaning of Ethnicity code (ETH_GROUP_LABEL)',
 'Meaning of Race code (RACE_GROUP_LABEL)',
 'Meaning of Veteran code (VET_GROUP_LABEL)',
 'Meaning of Years in business code (YIBSZFI_LABEL)',
 'Number of employer firms (FIRMPDEMP)',
 'Sales, value of shipments, or revenue of employer firms ($1,000) (RCPPDEMP)',
 'Number of employees (EMP)',
 'Annual payroll ($1,000) (PAYANN)'

Columns to remove:

 'Year (YEAR)',
 'Relative standard error of employer firms (%) (FIRMPDEMP_S)',
 'Relative standard error of sales, value of shipments, or revenue of employer firms (%) (RCPPDEMP_S)',
 'Relative standard error of number of employees (%) (EMP_S)',
 'Relative standard error of annual payroll (%) (PAYANN_S)'

In [16]:
# list(ReceiptSizeOfFirmDF.columns)

Column to merge from ReceiptSizeOfFirmDF:

 'Meaning of Sales, value of shipments, or revenue size of firms code (RCPSZFI_LABEL)'

In [15]:
# list(EmploymentSizeOfFirmDF.columns)

Column to merge from EmploymentSizeOfFirmDF:

 'Meaning of Employment size of firms code (EMPSZFI_LABEL)'

## Cleaning

- Columns we aren't planning to use
- Rename Columns we are planning to use
- Merge